**Imports**

In [1]:
import numpy as np
import timeit
from numpy import linalg
from numpy import array
from numpy.linalg import norm 

**Setup Aufgabe 42**

In [2]:
#Objects

#(Dimensions:n, Controller:q), (N-array, Controller:v1,v2..)
n = [4,8,16,32,64]
q = n[0]
#Matrices
A_tilde = np.random.randint(10,size=(q,q))
I = np.identity(q)
A = np.hstack((A_tilde,I))
N = [0,1]
v0 = N[0]
v1 = N[1]

#Vectors and maschine ε
b = np.ones((1,q),dtype=float)
c = (-1)* np.concatenate((np.ones(q,dtype=float),np.zeros(q,dtype=float)))
ε = np.finfo(float).eps

#Scaling
c_unit = c/norm(c)
b_unit = b/norm(b)

In [3]:
print("The norm of b is:",norm(b_unit),"The norm of c is:",norm(c_unit))

The norm of b is: 1.0 The norm of c is: 1.0


**Predictor-Corrector Algorithm of Mehrotra:**
* Divided into steps




**Step 1)**

In [4]:
k = 0
x_0 = (np.ones((2*q))**k)
λ_0 = (np.zeros(q)**k)
s_0 = (np.ones(2*q)**k)
t = (x_0,λ_0,s_0)
ε_0 = 10**-4
M_0 = 10**5

In [5]:
if (np.any(x_0)>0 and np.any(s_0)>0 and ε_0>0 and M_0>0 and k==0):
    print("We choose a starting point(x**0,λ**0,s**0) :=",t,
      "\n","with x**0 > 0:","check x_0:",x_0,
      "\n","and s**0 > 0:","check s_0:",s_0,
      "\n","ε_0>0:",ε_0,
      "\n","a large upper bound M>>0:=""check M:",M_0,
      "\n","and set k:=0","check k:",k)

We choose a starting point(x**0,λ**0,s**0) := (array([1., 1., 1., 1., 1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1., 1., 1., 1., 1.])) 
 with x**0 > 0: check x_0: [1. 1. 1. 1. 1. 1. 1. 1.] 
 and s**0 > 0: check s_0: [1. 1. 1. 1. 1. 1. 1. 1.] 
 ε_0>0: 0.0001 
 a large upper bound M>>0:=check M: 100000 
 and set k:=0 check k: 0


**Step 2)**

In [6]:
x = x_0**k
λ = λ_0**k
s = s_0**k
t1 = (x,λ,s)
μ = (x.T*s)/(q)

In [7]:
print("Set (x,λ,s):=",t1,
      "\n","and μ:=",μ)

Set (x,λ,s):= (array([1., 1., 1., 1., 1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1., 1., 1., 1., 1.])) 
 and μ:= [0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25]


**Step 3)**

In [8]:
in1 = np.dot(A,x)-b_unit
in2 = np.dot(A.T,λ)+s-c_unit
n1= norm(in1)
n2 = norm(in2)

In [9]:
if (n1<ε_0 and n2<ε_0 and μ<ε_0):
    print("If ||Ax-b||<ε is True:",n1<ε_0,
    "\n","||A^Tλ+s-c||<ε is True:",n2<ε_0,
    "\n","and μ<ε is True",μ<ε_0,
    "\n","STOP",
    "\n","The element (x,λ,s) is a good approximation of an optimal solution",)
else:
    print("Go to Step 4")


Go to Step 4


**Step 4**

In [10]:
n3 = norm(x)
n4 = norm(s)

In [11]:
if (n3>M_0 or n4>0):
    print("If ||x||>M:",n3>M_0,"or ||s||>M:",n4>0,
    "\n","STOP",
    "\n","The Problem either has no feasible solution, or it is ill conditioned")
else:
    print("Go to Step 5")
    

If ||x||>M: False or ||s||>M: True 
 STOP 
 The Problem either has no feasible solution, or it is ill conditioned


**Step 5)**

**Setup**
* Solve for R_hs

In [12]:
dx = np.gradient(x_0)**v0
dλ = np.gradient(λ_0)**v0
ds = np.gradient(s_0)**v0
t_2 = (dx,dλ,ds)
X = np.diag(np.ones(q))
S = np.diag(np.ones(q))
e = np.ones((q))
r1 = np.dot(A.T,λ_0)+s-c_unit
r2 = np.dot(A,x)-b_unit
r3 = np.dot(X,S)*e
R_hs = (-r1,-r2,-r3)

In [13]:
print("(dx,dλ,ds) are:",dx,dλ,ds)
print(R_hs)

(dx,dλ,ds) are: [1. 1. 1. 1. 1. 1. 1. 1.] [1. 1. 1. 1.] [1. 1. 1. 1. 1. 1. 1. 1.]
(array([-25.5, -18.5, -20.5, -10.5,  -2. ,  -2. ,  -2. ,  -2. ]), array([[-26.5, -15.5,  -9.5, -19.5]]), array([[-1., -0., -0., -0.],
       [-0., -1., -0., -0.],
       [-0., -0., -1., -0.],
       [-0., -0., -0., -1.]]))


**Step 6**

In [14]:
f1 = np.divide(x,dx**v0)
f2 = np.divide(s,ds**v0)
f1_min = np.min(f1)
f2_min = np.min(f2)
try:
    print(np.min(f1))
    print(np.min(f2))
except ZeroDivisionError:
    print("Division by zero") 

α_x = np.min([1,-f1_min])
α_s = np.min([1,-f2_min])
print(α_x,α_s)

1.0
1.0
-1.0 -1.0


In [15]:
print("Determine the maximum Step size along dx**N and ds**N",α_x,α_s)

Determine the maximum Step size along dx**N and ds**N -1.0 -1.0


**Step 7**

In [16]:
μ_pN = np.divide((x+α_x**v1*dx**v1)+(s+α_s**v1*ds**v1),v1)
σ = (μ_pN/μ)**3

In [17]:
print("Set μ_pN:=",μ[0:q],
     "\n","and σ:=",σ[0:q])

Set μ_pN:= [0.25 0.25 0.25 0.25] 
 and σ:= [0. 0. 0. 0.]


**Step 8)**

In [18]:
dX=np.diag(dx[0:q]**v0)
dS=np.diag(ds[0:q]**v0)
r4 = (X*S*e-μ[0:q]*σ[0:q]*e+dX[0:q]**v0*dS[0:q]**v0*e)

In [19]:
print("Solve for r4:=",r4,
     "\n","Determine:=",(dX,dS))

Solve for r4:= [[2. 1. 1. 1.]
 [1. 2. 1. 1.]
 [1. 1. 2. 1.]
 [1. 1. 1. 2.]] 
 Determine:= (array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]]), array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]]))


**Step 9)**

In [20]:
C = [0,1]
c0 = C[0]
c1 = C[1]
ηk = np.linspace(0.8,1,endpoint=False)
f3 = np.divide(x,dx**c0)
f4 = np.divide(s,ds**c0)
f3_min = np.array(np.min(f3))
f4_min = np.array(np.min(f4))
α_xC = np.min([1,0.8*all(-f3)])
α_sC = np.min([1,0.8*all(-f4)])

In [21]:
print("Choose a dampling parameter ηk∈[0.8,1) and calculate the step sizes dx**C and ds**C",(α_xC,α_sC))

Choose a dampling parameter ηk∈[0.8,1) and calculate the step sizes dx**C and ds**C (0.8, 0.8)


**Step 10**

In [22]:
x_k1 = x**k+α_xC*dx**c0
λ_k1 = λ**k+α_sC**c0*dλ**c0
s_k1 = s**k+α_sC**c0*ds**c0
k=k+1

In [23]:
print("Set x_k1:=",x_k1,
      "\n","and (λ**(k+1),s**(k+1)):=",(λ_k1,s_k1),
      "\n","k = k+1 :=",k)

Set x_k1:= [1.8 1.8 1.8 1.8 1.8 1.8 1.8 1.8] 
 and (λ**(k+1),s**(k+1)):= (array([2., 2., 2., 2.]), array([2., 2., 2., 2., 2., 2., 2., 2.])) 
 k = k+1 := 1


In [24]:
class predictor_corrector:
    def __init__(self,A,b,c,t1,ε,M_0):
        self.A = A
        self.b = b
        self.c = c
        self.t1 = t1
        self.ε = ε
        self.M = M_0
        print(t1)
        
l = predictor_corrector(A,b,c,t,ε,M_0)
print(l)

(array([1., 1., 1., 1., 1., 1., 1., 1.]), array([1., 1., 1., 1.]), array([1., 1., 1., 1., 1., 1., 1., 1.]))


**Overview**
* Recap:

In [25]:
print("The b vector is:", b,"∈ R^(n)",
      "\n","The vector c is:", c,"∈ R^(2n)",
      "\n","A_tilde is :","\n",A_tilde,
      "\n","The Identity Matrix:","\n",I,
      "\n","The Combined Matrix A is:","\n",A,"∈ R^(2n)",
      "\n","Unit vectors (b,c):=", (b_unit,c_unit),
      "\n","their sum is:", (sum(b_unit),sum(c_unit)),
      "\n","The vectors x_0 and s_0 are:",(x_0,s_0),"∈ R^(2n)",
      "\n","They are equal in each component:",(x_0==s_0),
      "\n","The vector λ_0 is:",λ_0,"∈ R^(n)",
      "\n","The tolerance ε_0 is equal to:", ε_0,
      "\n","The Majorant M_0 is assigned to:",M_0,
      "\n","We choose the starting point (x**0,λ**0,s**0) :=",t,"and did all checks",
      "\n","We then Set (x,λ,s):=",t1,
      "\n","and μ:=",μ,
      "\n","as all components of",(n1,n2,μ),
      "\n","are bigger then ε_0 we proceed to step 4 as the element (x,λ,s):",
      "\n",(x,λ,s),
      "\n","We concluded that is not a good approximation of an optimal solution",
      "\n","Since ||s||>M_0 we STOP here and can conclude that the Problem either has no feasible solution, or it is ill conditioned") 
        

The b vector is: [[1. 1. 1. 1.]] ∈ R^(n) 
 The vector c is: [-1. -1. -1. -1. -0. -0. -0. -0.] ∈ R^(2n) 
 A_tilde is : 
 [[7 9 9 1]
 [9 0 6 0]
 [0 5 1 3]
 [8 3 3 5]] 
 The Identity Matrix: 
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]] 
 The Combined Matrix A is: 
 [[7. 9. 9. 1. 1. 0. 0. 0.]
 [9. 0. 6. 0. 0. 1. 0. 0.]
 [0. 5. 1. 3. 0. 0. 1. 0.]
 [8. 3. 3. 5. 0. 0. 0. 1.]] ∈ R^(2n) 
 Unit vectors (b,c):= (array([[0.5, 0.5, 0.5, 0.5]]), array([-0.5, -0.5, -0.5, -0.5, -0. , -0. , -0. , -0. ])) 
 their sum is: (array([0.5, 0.5, 0.5, 0.5]), -2.0) 
 The vectors x_0 and s_0 are: (array([1., 1., 1., 1., 1., 1., 1., 1.]), array([1., 1., 1., 1., 1., 1., 1., 1.])) ∈ R^(2n) 
 They are equal in each component: [ True  True  True  True  True  True  True  True] 
 The vector λ_0 is: [1. 1. 1. 1.] ∈ R^(n) 
 The tolerance ε_0 is equal to: 0.0001 
 The Majorant M_0 is assigned to: 100000 
 We choose the starting point (x**0,λ**0,s**0) := (array([1., 1., 1., 1., 1., 1., 1., 1.]), array([1.,

**Time checks**


In [26]:
#Scaling
%timeit (b_unit,c_unit)
#Choosing the starting point
%timeit (t)
#Vector and Matrix construction
%timeit (A_tilde,A,I,x_0,λ_0,s_0,b,c)
%timeit (X,S,dX,dS)
#Calculations 
%timeit (R_hs,r4,f3,f4,f3_min,f4_min,α_xC,α_sC,x_k1,λ_k1,s_k1,k)
#Norms
%timeit (n1,n2,n3,n4)
#Gradient
%timeit (t_2)

121 ns ± 1.09 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
32.6 ns ± 0.585 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
260 ns ± 1.63 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
166 ns ± 0.906 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
339 ns ± 0.979 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
170 ns ± 1.78 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
30.4 ns ± 0.112 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [27]:
z = c.T*x
b1 = np.dot(A[0:,4:],x[4:])

**Refactorization start wip**

In [28]:
print("Thats z:",z,
      "\n","Thats the Matrix vector Product:",b1,
      "\n","Thats b:",b,
      "\n","Thats x:",x,
      "\n","Thats S:",S)

Thats z: [-1. -1. -1. -1. -0. -0. -0. -0.] 
 Thats the Matrix vector Product: [1. 1. 1. 1.] 
 Thats b: [[1. 1. 1. 1.]] 
 Thats x: [1. 1. 1. 1. 1. 1. 1. 1.] 
 Thats S: [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [29]:
def pc_alg(A,b,c,t,s,ε,M):
    return(x,λ,s)

In [30]:
pc_alg(A,b,c,t,s,ε,M_0)

(array([1., 1., 1., 1., 1., 1., 1., 1.]),
 array([1., 1., 1., 1.]),
 array([1., 1., 1., 1., 1., 1., 1., 1.]))

* Scaling starter Matrix

In [31]:
A_scale = [np.random.randint(10,size=(n[k],n[k]))for k in range (n[k-1])]
I_scale = [np.identity(n[k])for k in range (n[k-1])]
A_col = np.hstack((A_scale[k],I_scale[k]))

In [32]:
print(A_scale[0].shape,A_scale[1].shape,A_scale[2].shape,A_scale[3].shape)
print(I_scale[0].shape,I_scale[1].shape,I_scale[2].shape,I_scale[3].shape,I_scale[1])

(4, 4) (8, 8) (16, 16) (32, 32)
(4, 4) (8, 8) (16, 16) (32, 32) [[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
